In [9]:
pip install pymysql


   ---------------------------------------- 0.0/45.0 kB ? eta -:--:--
   --------- ------------------------------ 10.2/45.0 kB ? eta -:--:--
   --------------------------- ------------ 30.7/45.0 kB 330.3 kB/s eta 0:00:01
   ---------------------------------------- 45.0/45.0 kB 369.5 kB/s eta 0:00:00


In [27]:
import pandas as pd
import pymysql
""""df = pd.read_csv('kaac_details.csv')
df = pd.read_csv('North_Bengal_Bus_details.csv')
df = pd.read_csv('Jammu_Kashmir_details.csv')
df = pd.read_csv('West_Bengal_Bus_details.csv')
df = pd.read_csv('Kadamba_Bus_details.csv')
df = pd.read_csv('Chandighar_Bus_details.csv')
df = pd.read_csv('Assam_Bus_details.csv')
df = pd.read_csv('Bihar.csv')
df = pd.read_csv('South_Bengal.csv')
df = pd.read_csv('Himachal.csv')"""

# List of CSV file path
csv_files = ['kaac_details.csv', 'North_Bengal_Bus_details.csv', 'Jammu_Kashmir_details.csv', 'West_Bengal_Bus_details.csv', 'Kadamba_Bus_details.csv',
            'Chandighar_Bus_details.csv', 'Assam_Bus_details.csv', 'Bihar.csv', 'South_Bengal.csv', 'Himachal.csv']

# Read and concatenate all CSVs
df_list = [pd.read_csv(file) for file in csv_files]
combined_df = pd.concat(df_list, ignore_index=True)

#Data Cleaning
combined_df['Price'] = combined_df['Price'].astype(str).str.replace('INR ', '', regex=False)
combined_df['Seat_Availability'] = combined_df['Seat_Availability'].astype(str).str.extract(r'(\d+)')
combined_df.dropna(inplace=True)

#Add ID column
combined_df.insert(0, 'id', range(1, len(combined_df) + 1))

#Save to new CSV
combined_df.to_csv("bus_routes.csv", index=False)

#MySQL Database Connection
connection = pymysql.connect(
    host='127.0.0.1',
    user='root',
    passwd='root',
    database='redbus'
)
cursor = connection.cursor()

#Generate MySQL schema from DataFrame dtypes
dtype_mapping = {
    'int64': 'INT',
    'float64': 'FLOAT',
    'object': 'TEXT'
}
columns_with_types = ", ".join(
    f"`{col}` {dtype_mapping[str(dtype)]}" for col, dtype in combined_df.dtypes.items()
) 

#Create Table
table_name = "bus_routes"
cursor.execute(f"DROP TABLE IF EXISTS {table_name}")
cursor.execute(f"CREATE TABLE {table_name} ({columns_with_types})")

#Insert data row by row
for _, row in combined_df.iterrows():
    placeholders = ', '.join(['%s'] * len(row))
    insert_query = f"INSERT INTO {table_name} VALUES ({placeholders})"
    cursor.execute(insert_query, tuple(row))
    connection.commit()

#Clean up
cursor.close()
connection.close()
